In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from pprint import pprint

In [ ]:
file_path = "Bootcamp_Project_1"
csv_reader = csv.reader(fileGoogle-Playstore-32K.csv)